In [73]:
import pandas as pd
import os
import time
import numpy as np
import arcgis
from datetime import datetime, timedelta

# 1. Load/Condition Data from GDB


With the prepwork done in Pro or Arcpy, we can use the ArcGIS Python API to easily load this data into a SpatialDataFrame, which will allow us to do any geospatial manipulations we need to do. In this example, we'll mostly use the ``pandas`` functionality

In [74]:
project_gdb = r'utah.gdb'
collisions_path = os.path.join(project_gdb,'collisions_joined')
road_features_path = os.path.join(project_gdb,'static_features')

## 1.1 Load Data

In [75]:
df = arcgis.SpatialDataFrame().from_featureclass(collisions_path)

In [76]:
df.head()

,OBJECTID,Join_Count,TARGET_FID,CRASHID,CRASHDATE,CRASHTIME,RouteName,BeginMilePost,RoadView,CrashSeverity,...,NUMBERVEHICLESINVOLVED,FIRSTHARMFULEVENTID,ROADWAYTYPECD,ROUTEDIRECTION,EXITNUMBER,RAMPID,Latitude,Longitude,segment_id,SHAPE
0,1,1,1,10401151,2010-10-15,19:56:00,0009P,11.64,http://168.178.125.111/virtualnavigator/udot/V...,2,...,2,Other Motor Vehicle in Transport,Mainline,P,None,None,4119389.0,298229.0,28425,"{'x': -12609547.622000001, 'y': 4466906.681199..."
1,2,1,2,10394697,2010-08-20,19:33:00,0009P,11.85,http://168.178.125.111/virtualnavigator/udot/V...,1,...,2,Other Motor Vehicle in Transport,Mainline,P,None,None,4119704.0,298349.0,28431,"{'x': -12609406.6849, 'y': 4467306.825099997, ..."
2,3,1,3,10346333,2010-03-27,21:30:00,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,...,1,Delineator Post,Mainline,P,None,None,4122320.0,299671.0,24677,"{'x': -12607827.8035, 'y': 4470640.231600001, ..."
3,4,1,4,10352753,2010-04-16,13:45:00,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,...,1,Overturn/Rollover,Mainline,P,None,None,4122320.0,299671.0,24677,"{'x': -12607827.8035, 'y': 4470640.231600001, ..."
4,5,1,5,10357506,2010-03-05,9:48:00,0009P,21.00,http://168.178.125.111/virtualnavigator/udot/V...,1,...,1,Animal - Wild,Mainline,P,None,None,4118637.0,309403.0,28601,"{'x': -12595519.862399999, 'y': 4466288.306199..."


In [77]:
rdf = arcgis.SpatialDataFrame().from_featureclass(road_features_path)

## 1.2 Remove some bad data

In [78]:
# We don't care about accidents that happened far from known major/minor roads
df = df.dropna(how='any',subset=['segment_id'])

# Convert some types
df['segment_id'] = df.segment_id.astype('int64')
rdf['segment_id'] = rdf.segment_id.astype('int64') 
rdf['station_id'] = rdf.station_id.astype('int64') # Weather statiion ID

## 1.3 Parse Dates

In [79]:
df['CRASHDATE'] = df['CRASHDATE'].astype('str')
df['CRASHTIME'] = df['CRASHTIME'].astype('str')

df['timestamp'] = pd.to_datetime(df['CRASHDATE']+' '+df['CRASHTIME'])

time_index = pd.DatetimeIndex(df.timestamp).floor('1h')
df['hour'] = time_index.hour
df['weekday'] = time_index.weekday
df['month'] = time_index.month
df.to_csv('training_data/collisions_new.csv')

## 1.4 Remove/Replace missing data

In [80]:
rdf.loc[pd.isna(rdf.speed_limit),'speed_limit'] = np.median(rdf.speed_limit)
rdf.loc[pd.isna(rdf.aadt),'aadt'] = 0.0 # AKA, unknown, zero, etc. This will help differentiate major/minor roadways

# 2. Expand Feature Set
Once the data is in a DataFrame, it's very easy to manipulate to add certain features. Here we will add a accident count field by doing some joins and aggregations

In [81]:
accident_counts_per_segment = df.groupby('segment_id').size().reset_index(name='accident_counts').set_index('segment_id')

In [82]:
rdf = rdf.set_index('segment_id').join(accident_counts_per_segment)

In [83]:
# If there was no accident, set value to 0, not NaN
rdf.loc[pd.isna(rdf.accident_counts),'accident_counts'] = 0.0

# 3. Select Features

## 3.1 Feature Names
We can do most of our exploratory data analysis in ArcGIS Pro to determine a set of candidate features. Once we build our training set, we can do further EDA to determine the best feature set. Here we select the candidate features that we found in Pro 

In [84]:
print(rdf.columns.tolist())

['OBJECTID_1', 'Join_Count', 'TARGET_FID', 'Join_Count_1', 'TARGET_FID_1', 'OBJECTID', 'pre_dir', 'street_type', 'suf_dir', 'one_way', 'speed_limit', 'surface_type', 'surface_width', 'aadt', 'sinuosity', 'euclidean_length', 'segment_length', 'at_intersection', 'near_billboard', 'road_orient_approx', 'near_major_road', 'station_id', 'SHAPE', 'accident_counts']


In [85]:
static_feature_names = [
    'pre_dir', 
    'street_type',
    'suf_dir', 
    'one_way', 
    'speed_limit', 
    'surface_type', 
    'surface_width', 
    'aadt', 
    'sinuosity',          # \
    'euclidean_length',   # |  These together define road curvature
    'segment_length',     # /
    'near_major_road', 
    'road_orient_approx', 
    'at_intersection', 
    'near_billboard',
    'accident_counts'
]
temporal_feature_names = [
    'hour',
    'weekday',
    'month'
]

## 3.2 Select static features, join to accidents records

In [86]:
features = df\
    .xs(['timestamp','segment_id']+temporal_feature_names,axis=1)\
    .set_index('segment_id')\
    .join(
        rdf.xs(static_feature_names+['station_id'],axis=1),
        how='left'
    ).reset_index()

In [87]:
features.head()

,segment_id,timestamp,hour,weekday,month,pre_dir,street_type,suf_dir,one_way,speed_limit,...,aadt,sinuosity,euclidean_length,segment_length,near_major_road,road_orient_approx,at_intersection,near_billboard,accident_counts,station_id
0,1,2016-11-26 10:13:00,10,5,11,N,,,0,40.0,...,16689.0,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,72572024127
1,1,2017-10-16 16:03:00,16,0,10,N,,,0,40.0,...,16689.0,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,72572024127
2,1,2010-01-16 15:26:00,15,5,1,N,,,0,40.0,...,16689.0,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,72572024127
3,1,2013-12-16 15:48:00,15,0,12,N,,,0,40.0,...,16689.0,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,72572024127
4,2,2015-09-04 21:10:00,21,4,9,N,,,0,40.0,...,16689.0,1.0,87.835598,87.835598,0,1.187714,1,0,7.0,72572024127


## 3.3 Index by timestamp/road

In [88]:
time_index = pd.DatetimeIndex(features.timestamp).floor('1h')
feature_timestamp_series = time_index.to_series()
segment_id = features.segment_id.astype('str')

# Index for fast lookup
features['fid'] = feature_timestamp_series.values.astype('str') + segment_id.values
features.set_index('fid',inplace=True)

# Floor to nearest hour
features['timestamp'] = feature_timestamp_series.values


# Set target value
features['target'] = 1

In [89]:
features.head(15)

,segment_id,timestamp,hour,weekday,month,pre_dir,street_type,suf_dir,one_way,speed_limit,...,sinuosity,euclidean_length,segment_length,near_major_road,road_orient_approx,at_intersection,near_billboard,accident_counts,station_id,target
fid,,,,,,,,,,,,,,,,,,,,,
2016-11-26T10:00:00.0000000001,1,2016-11-26 10:00:00,10,5,11,N,,,0,40.0,...,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,72572024127,1
2017-10-16T16:00:00.0000000001,1,2017-10-16 16:00:00,16,0,10,N,,,0,40.0,...,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,72572024127,1
2010-01-16T15:00:00.0000000001,1,2010-01-16 15:00:00,15,5,1,N,,,0,40.0,...,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,72572024127,1
2013-12-16T15:00:00.0000000001,1,2013-12-16 15:00:00,15,0,12,N,,,0,40.0,...,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,72572024127,1
2015-09-04T21:00:00.0000000002,2,2015-09-04 21:00:00,21,4,9,N,,,0,40.0,...,1.0,87.835598,87.835598,0,1.187714,1,0,7.0,72572024127,1
2011-10-20T06:00:00.0000000002,2,2011-10-20 06:00:00,6,3,10,N,,,0,40.0,...,1.0,87.835598,87.835598,0,1.187714,1,0,7.0,72572024127,1
2016-09-14T07:00:00.0000000002,2,2016-09-14 07:00:00,7,2,9,N,,,0,40.0,...,1.0,87.835598,87.835598,0,1.187714,1,0,7.0,72572024127,1
2013-03-07T20:00:00.0000000002,2,2013-03-07 20:00:00,20,3,3,N,,,0,40.0,...,1.0,87.835598,87.835598,0,1.187714,1,0,7.0,72572024127,1
2015-04-16T14:00:00.0000000002,2,2015-04-16 14:00:00,14,3,4,N,,,0,40.0,...,1.0,87.835598,87.835598,0,1.187714,1,0,7.0,72572024127,1


# 4 Negative Sampling
We have a large set of positive examples, that is, when accidents occured. Any time/segment when an accident didn't occur is a valid negative sample. We sample possible negtive examples by sampling from and mutating the positive examples until they aren't positive. We do this by changing either the hour, day or segment ID.

In [90]:
class NegativeSampler:
    """
    Given features (positive examples), roads, times and the number of samples to draw, draws a sample
    """
    def __init__(self,num_samples,
                 accidents,roads,
                 static_feature_names,
                 seed=None):

        self.roads = roads
        self.accidents = accidents
        self.num_samples = num_samples
        
        self.static_feature_names = static_feature_names
        
        if seed:
            np.random.seed(seed)
    
    def sample(self):
        segment_ids = self.roads.index.to_series()
        
        altered = pd.DataFrame()
        num_to_sample = self.num_samples
        
        while num_to_sample > 0:
            samples = self.accidents.sample(n=num_to_sample,replace=True).reset_index()
            alt = self._mod(samples.copy(),segment_ids)
            # Create an index
         
            ats = pd.DatetimeIndex(alt.timestamp).floor('1h').to_series().values.astype('str')
            asegid = alt.segment_id.astype('str')
            
            # Set index on altered samples
            alt['fid'] = ats + asegid.values
            alt.set_index('fid',inplace=True)
            
            altered = altered.append(alt)

            # Which happened before? They shouldn't get negative samples
            intersection = altered.index.intersection(self.accidents.index)
            idxer = intersection.get_values()
            
            # Drop samples where accidents occurred.
            altered = altered.drop(idxer)
            
            num_to_sample = self.num_samples - altered.shape[0]
            
        
        altered['target'] = 0
        
        ts = pd.DatetimeIndex(altered.timestamp)
        segment_ids = altered.segment_id
        
        altered = altered.reset_index()
        
        station_ids = self.roads.loc[segment_ids].station_id
        static_features = self.roads.loc[segment_ids][self.static_feature_names]
        
        altered[self.static_feature_names] = static_features.values
        altered['station_id'] = station_ids.values
        altered['weekday'] = ts.weekday
        altered['month'] = ts.month
        
        altered = altered.drop(columns=['fid'])
        return altered
    
    def _mod(self,samples,segment_ids):
        # Get the current timestamps
        ts = pd.DatetimeIndex(samples.timestamp)
        
        # Hour, Day, Year
        hour = ts.hour.to_series()
        day = ts.dayofyear.to_series()
        year = ts.year.to_series()

        # Road ID
        segment_id = samples.segment_id.copy()
        
        # Index of samples to mutate
        feat_i = np.random.randint(0,3,size=samples.shape[0])

        ##########################
        # i == 0
        # Change hour of day
        idx = feat_i == 0
        samp_i = samples.loc[idx]
        N = samp_i.shape[0]

        # Sample until we have all different hours of the day
        num_same = N
        new_hours = hour[idx].values.copy()
        dif_idx = np.ones(N,dtype='bool')
        while num_same != 0:
            new_hours[dif_idx] = np.random.choice(24,size=num_same)
            dif_idx = new_hours == hour[idx]
            num_same = dif_idx.sum()

        # Create new timestamps
        new_timestamps = year[idx].apply(pd.Timestamp,args=(1,1))
        new_timestamps = pd.DatetimeIndex(new_timestamps) 
        new_timestamps += pd.TimedeltaIndex(day[idx]-1,unit='D') # same day
        new_timestamps += pd.TimedeltaIndex(new_hours,unit='H') # new hour
        
        
        samples.loc[idx,'hour'] = new_hours
        samples.loc[idx,'segment_id'] = segment_id[idx]
        samples.loc[idx,'timestamp'] = new_timestamps.copy()
        ##########################

        ##########################
        # i == 1
        # Change day of year
        idx = feat_i == 1
        samp_i = samples.loc[idx]
        N = samp_i.shape[0]

        is_leap_yr = ts[idx].is_leap_year

        # Sample until we have all different days of the year.
        num_same = N
        new_days = day[idx].values.copy()
        dif_idx = np.ones(N,dtype='bool')
        while num_same != 0:
            # Pay attention to leap years
            dif_leap_yr = (dif_idx&is_leap_yr)
            dif_no_leap_yr = dif_idx&(~is_leap_yr)
            
            new_days[dif_leap_yr] = np.random.choice(np.arange(1,367,dtype='int'),size=dif_leap_yr.sum())
            new_days[dif_no_leap_yr] = np.random.choice(np.arange(1,366,dtype='int'),size=dif_no_leap_yr.sum())

            dif_idx = new_days == day[idx]
            num_same = dif_idx.sum()
        
        # Create new timestamps
        timestamps = year[idx].apply(pd.Timestamp,args=(1,1))
        new_timestamps = pd.DatetimeIndex(timestamps) 
        new_timestamps += pd.TimedeltaIndex(new_days,unit='D')  # new day
        new_timestamps += pd.TimedeltaIndex(hour[idx],unit='H') # same hour

        samples.loc[idx,'hour'] = new_timestamps.hour
        samples.loc[idx,'segment_id'] = segment_id[idx]
        samples.loc[idx,'timestamp'] = new_timestamps.copy()
        ##########################

        ##########################
        # i == 2
        # Change road
        idx = feat_i == 2
        samp_i = samples.loc[idx]
        N = samp_i.shape[0]

        num_same = N
        new_roads = segment_id[idx].values.copy()
        dif_idx = np.ones(N,dtype='bool')

        while num_same != 0:
            new_roads[dif_idx] = segment_ids.sample(n=num_same,replace=True).values
            dif_idx = new_roads == segment_id[idx]
            num_same = dif_idx.sum()

        samples.loc[idx,'segment_id'] = new_roads
        samples.loc[idx,'hour'] = hour[idx].values
        samples.loc[idx,'timestamp'] = ts[idx].copy()
        
        samples['segment_id'] = samples['segment_id'].astype('int64')     
        return samples

In [91]:
t = time.time()
N = features.shape[0] * 4
ns = NegativeSampler(N,features,rdf,static_feature_names,seed=12345)
negative_examples = ns.sample().set_index(['timestamp','segment_id','station_id'])

print((time.time() - t),'seconds')

39.565378189086914 seconds


In [92]:
if not os.path.exists('training_data'):
    os.mkdir('training_data')

In [93]:
negative_examples.to_csv('training_data/utah_negative_examples.csv')

In [94]:
negative_examples.head()

,,,hour,weekday,month,pre_dir,street_type,suf_dir,one_way,speed_limit,surface_type,surface_width,aadt,sinuosity,euclidean_length,segment_length,near_major_road,road_orient_approx,at_intersection,near_billboard,accident_counts,target
timestamp,segment_id,station_id,,,,,,,,,,,,,,,,,,,,
2014-01-11 17:00:00,23524,72575524101,17,5,1,,,,1,25.0,PAVED,0,3183.0,1.010711,721.107956,728.831980,0,2.124443,0,0,35.0,0
2014-10-09 20:00:00,5753,72572024127,20,3,10,S,,E,0,35.0,,0,11763.0,1.000000,74.051639,74.051639,0,1.562353,0,0,3.0,0
2015-08-19 08:00:00,141302,72570594030,8,2,8,,RD,,0,25.0,,18,0.0,1.000042,418.596688,418.614378,0,1.240349,0,0,0.0,0
2014-11-12 13:00:00,153026,72572424174,13,2,11,W,,S,0,25.0,,0,0.0,1.000000,197.361685,197.361685,0,0.018032,1,0,28.0,0
2017-01-27 16:00:00,44435,72572424174,16,4,1,E,,,0,65.0,,0,20191.0,1.076068,2629.327473,2829.333868,0,0.488510,1,0,140.0,0


In [95]:
# Reselect Features
positive_examples = df\
    .xs(['segment_id']+temporal_feature_names,axis=1)\
    .set_index('segment_id')\
    .join(
        rdf.xs(static_feature_names+['station_id'],axis=1),
        how='left'
    ).reset_index()
    
positive_examples = positive_examples.set_index([time_index.floor('1h'),'segment_id','station_id'])
positive_examples['target'] = 1
positive_examples.head()

,,,hour,weekday,month,pre_dir,street_type,suf_dir,one_way,speed_limit,surface_type,surface_width,aadt,sinuosity,euclidean_length,segment_length,near_major_road,road_orient_approx,at_intersection,near_billboard,accident_counts,target
timestamp,segment_id,station_id,,,,,,,,,,,,,,,,,,,,
2016-11-26 10:00:00,1,72572024127,10,5,11,N,,,0,40.0,,0,16689.0,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,1
2017-10-16 16:00:00,1,72572024127,16,0,10,N,,,0,40.0,,0,16689.0,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,1
2010-01-16 15:00:00,1,72572024127,15,5,1,N,,,0,40.0,,0,16689.0,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,1
2013-12-16 15:00:00,1,72572024127,15,0,12,N,,,0,40.0,,0,16689.0,1.0,110.198977,110.198977,0,1.022803,1,0,4.0,1
2015-09-04 21:00:00,2,72572024127,21,4,9,N,,,0,40.0,,0,16689.0,1.0,87.835598,87.835598,0,1.187714,1,0,7.0,1


In [96]:
positive_examples.to_csv('training_data/utah_positive_examples.csv')

In [97]:
rdf.to_csv('training_data/road_features.csv')

In [98]:
df.to_csv('training_data/collisions_new.csv')